## Imports

In [44]:
# Default
import sys
import datetime

sys.path.append('../../da-irl')

# Local
from src.impl.activity_env import ActivityEnv
from src.impl.activity_mdp import ATPTransition, ActivityMDP
from src.impl.activity_params import MATSimParameters
from src.impl.activity_rewards import ActivityRewardFunction
from src.irl.meirl import MaxEntIRLAgent
from src.misc.utils import create_dir_if_not_exists
from src.file_io.trace_loader import TraceLoader

# Vendor
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from pandas.tseries.offsets import Day
import shapely
from shapely.geometry import box
import numpy as np
import folium

shapely.speedups.enable()



In [5]:
# Global magics statements

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Constants and Utility Functions

In [759]:
# BBOX:
LAT_MIN=32.831
LAT_MAX=38.67
LNG_MIN=-128.9
LNG_MAX=-117.41

# DOW
MON = 0
TUES = 1
WEDS = 2
THUR = 3
FRI = 4
SAT = 5
SUN = 6

# Path
PATH = '../../da-irl/data/traces/traces_persona_1.csv'

def great_circle_distance(pt1,pt2):
    """
    Return the great-circle distance in kilometers between two points,
    defined by a tuple (lat, lon).
    Examples
    --------
    >>> brussels = (50.8503, 4.3517)
    >>> paris = (48.8566, 2.3522)
    >>> great_circle_distance(brussels, paris)
    263.9754164080347
    """
    r = 6371.
    
    delta_latitude = np.radians(pt1[:,0] - pt2[:,0])
    delta_longitude = np.radians(pt1[:,1] - pt2[:,1])
    latitude1 = np.radians(pt1[:,0])
    latitude2 = np.radians(pt2[:,0])

    a = np.sin(delta_latitude / 2) ** 2 + np.cos(latitude1) * np.cos(latitude2) * np.sin(delta_longitude / 2) ** 2
    return r * 2. * np.arcsin(np.sqrt(a))


In [760]:
import math
from datetime import timedelta
from geopy.distance import great_circle
"""
INPUTS:
    df={o1,o2,...,on} Set of objects
    spatial_threshold = Maximum geographical coordinate (spatial) distance value
    temporal_threshold = Maximum non-spatial distance value
    min_neighbors = Minimun number of points within Eps1 and Eps2 distance
OUTPUT:
    C = {c1,c2,...,ck} Set of clusters
"""
def ST_DBSCAN(df, spatial_threshold, temporal_threshold, min_neighbors):
    cluster_label = 0
    NOISE = -1
    UNMARKED = 777777
    stack = []

    # initialize each point with unmarked
    df['cluster'] = UNMARKED
    
    # for each point in database
    for index, point in df.iterrows():
        if df.loc[index]['cluster'] == UNMARKED:
            neighborhood = retrieve_neighbors(index, df, spatial_threshold, temporal_threshold)
            
            if len(neighborhood) < min_neighbors:
                df.set_value(index, 'cluster', NOISE)

            else: # found a core point
                cluster_label = cluster_label + 1
                df.set_value(index, 'cluster', cluster_label)# assign a label to core point

                for neig_index in neighborhood: # assign core's label to its neighborhood
                    df.set_value(neig_index, 'cluster', cluster_label)
                    stack.append(neig_index) # append neighborhood to stack
                
                while len(stack) > 0: # find new neighbors from core point neighborhood
                    current_point_index = stack.pop()
                    new_neighborhood = retrieve_neighbors(current_point_index, df, spatial_threshold, temporal_threshold)
                    
                    if len(new_neighborhood) >= min_neighbors: # current_point is a new core
                        for neig_index in new_neighborhood:
                            neig_cluster = df.loc[neig_index]['cluster']
                            if (neig_cluster != NOISE) & (neig_cluster == UNMARKED): 
                                # TODO: verify cluster average before add new point
                                df.set_value(neig_index, 'cluster', cluster_label)
                                stack.append(neig_index)
    return df


def retrieve_neighbors(index_center, df, spatial_threshold, temporal_threshold):
    neigborhood = []

    center_point = df.loc[index_center]

#     # filter by time 
#     min_time = center_point['start_time'] - timedelta(minutes = temporal_threshold)
#     max_time = center_point['end_time'] + timedelta(minutes = temporal_threshold)
#     df = df[(df['start_time'] >= min_time) & (df['end_time'] <= max_time)]
    
    # filter by distance
    for index, point in df.iterrows():
        if index != index_center:
            distance = great_circle((center_point['lat'], center_point['lng']), (point['lat'], point['lng'])).meters
            
            if distance <= spatial_threshold:
#                 print distance
                neigborhood.append(index)

    return neigborhood

## Filtering

In [761]:

class User:
    def __init__(self,records_path):
        self._records_path = records_path
        self._records = None
        
        self.name = None
        self.home = None
        self.work = None
        self.weekend_days = [SAT,SUN]  # hard code for now
        self.work_start_hr = 13 # hard code for now
        self.work_end_hr = 16 # hard code for now
        self.early_morning_hr = 5
        self.late_night_hr = 18
        
        
        self.attributes = {}
        self._load_records()
    
    def _load_records(self):
        df = TraceLoader.load_traces_from_csv(self._records_path)
        
        df=self._process_home_work_other(df)
        df=self._process_speed(df)
        df=self._process_duration(df)
        self._records=df
        
    def _process_home_work_other(self,df):
        df['at_home']=0
        df['at_work']=0
        df['at_other']=0

        for idx,row in df.iterrows():
            tup = (row.enter_time.dayofweek, row.enter_time.hour, row.exit_time.dayofweek, row.exit_time.hour)
            is_at_home = self._filter_home(tup)
            is_at_work = self._filter_work(tup)
            is_at_other = ((not is_at_home) & (not is_at_work))
            df.loc[idx,'at_home'] = 1 if is_at_home else 0
            df.loc[idx,'at_work'] = 1 if is_at_work else 0
            df.loc[idx,'at_other'] = 1 if is_at_other else 0
        df=df.dropna()
        return df
        
    def _filter_home(self, tup):
        enter_day,enter_hr,exit_day,exit_hr = tup
        is_same_day = (enter_day==exit_day)
        is_morning_hour = (exit_hr <= self.early_morning_hr)
        is_consecutive_day = (((enter_day==SUN) & (exit_day==MON))|(enter_day + 1 == exit_day))
        is_late_night = ((enter_hr >= self.late_night_hr)&is_morning_hour)
        cond_1 = (is_consecutive_day & is_late_night)
        cond_2 = (is_same_day & is_morning_hour)
        return cond_1|cond_2
    
    def _filter_work(self, tup):
        enter_day,enter_hr,exit_day,exit_hr = tup
        is_work_hour = ((enter_hr>self.work_start_hr)&(exit_hr<self.work_end_hr))
        is_work_day = (enter_day not in self.weekend_days)&(exit_day not in self.weekend_days)
        is_same_day = (enter_day==exit_day)
        return is_work_hour & (is_work_day & is_same_day)
    
    def _process_speed(self,df):
        cons_pts = np.array(zip(zip(df.lat.values[0:],df.lng.values[0:]),zip(df.lat.values[1:],df.lng.values[1:])))
        pts1=cons_pts[:,0]
        pts2=cons_pts[:,1]
        dist=np.round(great_circle_distance(pts1,pts2),5)
        df['dist'] = np.hstack([[0],dist])
        
        enters=df.enter_time.dt.to_pydatetime()
        exits=df.exit_time.dt.to_pydatetime()
        time_diff = np.diff(zip(exits[0:],enters[1:]))
        time_diff_s = np.apply_along_axis(lambda x: x[0].seconds,1, time_diff)
        df['time_diff']=np.vstack([[0],time_diff])
        df['speed'] = np.hstack([[0],np.round(dist/time_diff_s,5)])
        return df
    
    def _process_duration(self,df):
        df['duration']=df.exit_time-df.enter_time
        return df
        

In [762]:
#         self._records = gpd.GeoDataFrame(df.drop(['lat', 'lng'], axis=1),
#                                 crs={'init': 'epsg:4326'},
#                                 geometry=[shapely.geometry.Point(xy) for xy in zip(df.lat, df.lng)])

In [763]:
df=User(PATH)._records

In [764]:
from pandas.tseries.offsets import *

In [765]:
df['start_day']=df.enter_time.dt.normalize()
df['start_time']=df.enter_time.dt.floor('15min')
df['end_time']=df.exit_time.dt.floor('15min')
df['end_day'] = start_day + Day(1)- DateOffset(minutes=15)

In [766]:
x=[]
idxs=[]
for i,row in df.iterrows():
    if row.end_day<row.end_time:
        x.append(row)
        idxs.append(i)
df.drop(df.index[idxs],inplace=True)
days=[]
for row in x:
    prev_day = row.copy()
    next_day = row.copy()
    next_day.end_time = prev_day.end_time
    prev_day.end_time = prev_day.end_day
    next_day.start_time = next_day.start_day
    days.append(pd.concat([prev_day,next_day],1).T)
df=df.append(pd.concat(days),ignore_index=True)
df=df.sort_values(by='start_time')
df=df.reindex(range(len(df)))

In [767]:
from sklearn.cluster import DBSCAN

In [768]:
db = DBSCAN(eps=0.001, min_samples=5)

In [769]:
df['cluster']=db.fit_predict(df[['lat','lng']].values)

In [782]:
activities = df.copy()
labels = df.cluster.values
max_label = np.max(labels)
for label in xrange(max_label + 1):
    home_count = np.sum(activities.loc[labels == label, 'at_home'])
    work_count = np.sum(activities.loc[labels == label, 'at_work'])

    if (home_count == 0) and (work_count == 0):
        continue
    elif home_count > work_count:
        activities.loc[(labels == label) & (activities.at_work != 1), 'at_home'] = 1
    elif home_count < work_count:
        activities.loc[(labels == label) & (activities.at_home != 1), 'at_work'] = 1

activities.loc[(activities.at_home == 0) & (activities.at_work == 0), 'at_other'] = 1
activities.loc[(activities.at_home==1)&(activities.at_other==1),'at_other']=0
activities.loc[(activities.at_work==1)&(activities.at_other==1),'at_other']=0

In [783]:
df=activities

In [784]:
gdf= gpd.GeoDataFrame(df.drop(['lat', 'lng'], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[shapely.geometry.Point(xy) for xy in zip(df.lat, df.lng)])

In [785]:
def get_current_label(row):
    if(row.at_home==1):
        return 'h'
    elif(row.at_work==1):
        return 'w'
    else:
        return 's'

def make_seg_series(start, end, label, freq='15min'):
    dr = pd.date_range(start, end,freq=freq)
    label_ser = np.array([label]*len(dr),dtype=object)
    return pd.Series(label_ser,index=dr)
    
def segment_day(df):
    segs = []
    # starting day w/ travel
    for idx,(day,row) in enumerate(df.iterrows()):
        if idx==0:
            start_time = row.start_day
            label='h'
        else:
            start_time = end_time
            label='car'
        end_time = row.start_time
        
        segs.append(make_seg_series(start_time,end_time,label))
        start_time = end_time
        label = get_current_label(row)
        
        end_time = row.end_time
        segs.append(make_seg_series(start_time, end_time, label))
        
        if (idx == len(df)) and (row.end_time<row.end_day):
            label = 'car'
            end_time = row.end_day
            segs.append(make_seg_series(start_time, end_time, label))
    return pd.concat(segs)

In [786]:
segs = []
df=activities.copy()
for d,grp in df.groupby([df.start_day.dt.year,df.start_day.dt.month,df.start_day.dt.day,df.start_day.dt.dayofweek]):
    if d[-1] not in [5,6]:
        seg=np.array(segment_day(grp).values)[:96]
        segs.append(seg)

In [790]:
traj=np.array(segs,dtype='S16')

ValueError: setting an array element with a sequence

In [791]:
np.save('../data/traces/trajectories/p0',traj)

In [734]:
%pwd

u'/Users/sfeygin/current_code/python/da-irl/notebooks'

In [779]:
traj

array([['h', 'h', 'h', ..., 'h', 'h', 'h'],
       ['h', 'h', 'h', ..., 'h', 'h', 'h'],
       ['h', 'h', 'h', ..., 'car', 'car', 'car'],
       ..., 
       ['h', 'h', 'h', ..., 'h', 'h', 'h'],
       ['h', 'h', 'h', ..., 'h', 'h', 'h'],
       ['h', 'h', 'h', ..., 'h', 'h', 'h']],
      dtype='|S16')